# Exploratory Data Analysis (EDA) - Olist Churn Prediction

This notebook performs comprehensive exploratory data analysis on the Olist Brazilian E-Commerce dataset.

## Objectives:
1. Load and understand the dataset structure
2. Analyze data quality and missing values
3. Explore customer behavior patterns
4. Visualize key metrics and distributions
5. Identify insights for feature engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Data Loading

In [ ]:
# Load datasets
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
order_items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')
payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')

print("Dataset Shapes:")
print(f"Customers: {customers_df.shape}")
print(f"Orders: {orders_df.shape}")
print(f"Order Items: {order_items_df.shape}")
print(f"Products: {products_df.shape}")
print(f"Payments: {payments_df.shape}")

## 2. Data Quality Assessment

In [ ]:
# Check missing values
def check_missing(df, name):
    print(f"\n{name} - Missing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({'Count': missing, 'Percentage': missing_pct})
    print(missing_df[missing_df['Count'] > 0].sort_values('Count', ascending=False))

check_missing(customers_df, "Customers")
check_missing(orders_df, "Orders")
check_missing(order_items_df, "Order Items")
check_missing(products_df, "Products")
check_missing(payments_df, "Payments")

## 3. Temporal Analysis

In [ ]:
# Convert timestamps
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])

# Time range
print(f"Data Range: {orders_df['order_purchase_timestamp'].min()} to {orders_df['order_purchase_timestamp'].max()}")
print(f"Total Days: {(orders_df['order_purchase_timestamp'].max() - orders_df['order_purchase_timestamp'].min()).days}")

# Orders over time
orders_df['year_month'] = orders_df['order_purchase_timestamp'].dt.to_period('M')
monthly_orders = orders_df.groupby('year_month').size()

plt.figure(figsize=(14, 6))
monthly_orders.plot(kind='line', marker='o')
plt.title('Orders Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Customer Analysis

In [ ]:
# Merge for customer analysis
df = pd.merge(orders_df, customers_df, on='customer_id', how='left')

# Unique customers
print(f"Total Orders: {len(df)}")
print(f"Unique Customers: {df['customer_unique_id'].nunique()}")
print(f"Average Orders per Customer: {len(df) / df['customer_unique_id'].nunique():.2f}")

# Orders per customer distribution
orders_per_customer = df.groupby('customer_unique_id').size()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(orders_per_customer, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Orders per Customer', fontweight='bold')
axes[0].set_xlabel('Number of Orders')
axes[0].set_ylabel('Number of Customers')
axes[0].set_yscale('log')

# Value counts
order_counts = orders_per_customer.value_counts().sort_index()
axes[1].bar(order_counts.index[:10], order_counts.values[:10], edgecolor='black', alpha=0.7)
axes[1].set_title('Top 10 Order Frequencies', fontweight='bold')
axes[1].set_xlabel('Number of Orders')
axes[1].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

print(f"\nCustomers with only 1 order: {(orders_per_customer == 1).sum()} ({(orders_per_customer == 1).sum() / len(orders_per_customer) * 100:.1f}%)")
print(f"Customers with 2+ orders: {(orders_per_customer >= 2).sum()} ({(orders_per_customer >= 2).sum() / len(orders_per_customer) * 100:.1f}%)")

## 5. Geographic Analysis

In [ ]:
# Top states
state_counts = df['customer_state'].value_counts().head(10)

plt.figure(figsize=(12, 6))
state_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Top 10 States by Number of Orders', fontsize=16, fontweight='bold')
plt.xlabel('State')
plt.ylabel('Number of Orders')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTotal States: {df['customer_state'].nunique()}")
print(f"Top 3 States: {', '.join(state_counts.head(3).index.tolist())}")

## 6. Product Analysis

In [ ]:
# Merge with products
df_items = pd.merge(order_items_df, products_df[['product_id', 'product_category_name']], on='product_id', how='left')

# Top categories
top_categories = df_items['product_category_name'].value_counts().head(15)

plt.figure(figsize=(12, 8))
top_categories.plot(kind='barh', color='coral', edgecolor='black')
plt.title('Top 15 Product Categories', fontsize=16, fontweight='bold')
plt.xlabel('Number of Items Sold')
plt.ylabel('Category')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTotal Product Categories: {df_items['product_category_name'].nunique()}")
print(f"Missing Categories: {df_items['product_category_name'].isnull().sum()} ({df_items['product_category_name'].isnull().sum() / len(df_items) * 100:.1f}%)")

## 7. Payment Analysis

In [ ]:
# Payment types
payment_types = payments_df['payment_type'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
axes[0].pie(payment_types.values, labels=payment_types.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Payment Type Distribution', fontweight='bold')

# Payment value distribution
axes[1].hist(payments_df['payment_value'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Payment Value Distribution', fontweight='bold')
axes[1].set_xlabel('Payment Value (R$)')
axes[1].set_ylabel('Frequency')
axes[1].set_xlim(0, 500)

plt.tight_layout()
plt.show()

print(f"\nPayment Statistics:")
print(payments_df['payment_value'].describe())

## 8. Price and Freight Analysis

In [ ]:
# Price and freight statistics
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Price distribution
axes[0].hist(order_items_df['price'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0].set_title('Product Price Distribution', fontweight='bold')
axes[0].set_xlabel('Price (R$)')
axes[0].set_ylabel('Frequency')
axes[0].set_xlim(0, 500)

# Freight distribution
axes[1].hist(order_items_df['freight_value'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('Freight Value Distribution', fontweight='bold')
axes[1].set_xlabel('Freight Value (R$)')
axes[1].set_ylabel('Frequency')
axes[1].set_xlim(0, 100)

plt.tight_layout()
plt.show()

print("\nPrice Statistics:")
print(order_items_df['price'].describe())
print("\nFreight Statistics:")
print(order_items_df['freight_value'].describe())

## 9. Churn Analysis Preparation

In [ ]:
# Calculate time between orders for repeat customers
customer_orders = df.groupby('customer_unique_id')['order_purchase_timestamp'].agg(['min', 'max', 'count']).reset_index()
customer_orders['days_between'] = (customer_orders['max'] - customer_orders['min']).dt.days

# Filter repeat customers
repeat_customers = customer_orders[customer_orders['count'] > 1]

plt.figure(figsize=(12, 6))
plt.hist(repeat_customers['days_between'], bins=50, edgecolor='black', alpha=0.7, color='purple')
plt.title('Days Between First and Last Order (Repeat Customers)', fontsize=16, fontweight='bold')
plt.xlabel('Days')
plt.ylabel('Number of Customers')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nRepeat Customer Statistics:")
print(repeat_customers['days_between'].describe())

## 10. Key Insights

### Summary:
1. **Dataset Size**: ~100k orders from ~99k unique customers
2. **Customer Behavior**: Majority of customers (>95%) make only 1 order - high churn risk
3. **Geographic Distribution**: Concentrated in SP, RJ, MG states
4. **Product Categories**: Diverse catalog with 70+ categories
5. **Payment Methods**: Credit card is dominant payment method
6. **Temporal Patterns**: Clear growth trend over time

### Recommendations for Feature Engineering:
- **RFM Features**: Recency, Frequency, Monetary value
- **Behavioral Features**: Order velocity, product diversity
- **Geographic Features**: State-based patterns
- **Payment Features**: Preferred payment method
- **Temporal Features**: Time since first purchase

In [ ]:
print("\n" + "="*50)
print("EDA COMPLETED SUCCESSFULLY!")
print("="*50)
print("\nNext Steps:")
print("1. Baseline Model Development")
print("2. Feature Engineering")
print("3. Model Optimization")
print("4. Model Evaluation")